In [1]:
import numpy as np
from sklearn.metrics import jaccard_similarity_score

In [67]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data/comoda_data.csv")

In [5]:
df = df.replace(-1.0, np.nan)
df = df.dropna(how="any")

## Clean Data Here

In [6]:
## CONSTANTS ASSUMPTION
sigma = 1
threshold = 0.1
columns = df.columns.tolist()
columns.remove("rating")
columns.append("rating")
df = df[columns]
X = df[columns[2:-1]]
y = df[columns[-1:]]
df = df.reset_index(drop=True)
#df

In [7]:
class neighbors:
    def __init__(self, X, y):
        self.model = KNeighborsClassifier(n_neighbors=10)
        self.model.fit(X,y)
    def get_neighbors_data(self, user, itemID):
        #all_neighbors_indices = self.model.kneighbors(user[2:-1])[1][0]
        #print(all_neighbors_indices)
        #neighbors_data = df.iloc[all_neighbors_indices]
        neighbors_data = df[(df.itemID == itemID)]
        #print(neighbors_data)
        self.good_neighbors = pd.DataFrame()
        #print(neighbors_data.shape)
        threshold = 0.3
        for index, row in neighbors_data.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], user[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.good_neighbors=self.good_neighbors.append(row, ignore_index=True)
                #print(row)
                #print(good_neighbors)
                #print("Inside Threshold")
            #print(self.good_neighbors)
        #print(self.good_neighbors)
        return self.good_neighbors

In [8]:
class similarity_class:
    def __init__(self):
        pass
    def jaccard_similarity(self, x, y):
        return jaccard_similarity_score(x,y)

In [60]:
class neighborhood_contribution:
    def __init__(self):
        pass
    def neighbor_rating(self, neighbor, itemID, sigma, threshold):
        self.ratings_sum=0
        self.similarity_sum=0
        #print(type(user))
        #print(user)
        #print(neighbor[2:10])
        #print(df.itemID)
        ratings = df[(df.userID == neighbor.userID) & (df.itemID == itemID)]
        #print(ratings.shape)
        for index, user_rating in ratings.iterrows():
            similarity = jaccard_similarity_score(neighbor[2:-1],user_rating[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.ratings_sum += user_rating.rating * similarity
                self.similarity_sum += similarity
                #print(self.similarity_sum)
            #print(self.ratings_sum)
            #print(neighbor[-10:],user_rating[-10:])
        #print("Rating sum")
        #print(self.ratings_sum)
        #print("Similarity Sum")
        #print(self.similarity_sum)
        try:
            rating = self.ratings_sum / self.similarity_sum
            #rating = ratings.rating.mean()
        except:
            rating = 0
        #print(rating)
        return rating
    def neighbor_average(self,neighbor,sigma,threshold):
        ratings_of_neighbor = df[df.userID == neighbor.userID]#.rating.mean()
        rating_sum=0;
        count=0;
        for index, row in ratings_of_neighbor.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], neighbor[2:-1])
            if similarity > threshold:
                rating_sum+=row.rating
                count+=1
        average_rating_in_given_context = rating_sum/count
        #print(average_rating_in_given_context)
        return average_rating_in_given_context
    def baseline_rating(self, itemID):
        average_item_rating = df[df.itemID == itemID].rating.mean()
        return average_item_rating

In [10]:
def numerator(neighborhood_contribution_rating, similarity_of_neighbors):
    numerator_sum=0
    #print(neighborhood_contribution_rating)
    #print(similarity_of_neighbors)
    for neighbor_rating, similarity in zip(neighborhood_contribution_rating,similarity_of_neighbors):
        numerator_sum+= neighbor_rating*similarity
    return numerator_sum

In [51]:
def predict_rating(user,itemID):
    similarity_function = similarity_class()
    user_neighbors = neighbors(X,y)
    user_neighborhood_contribution = neighborhood_contribution()
    neighbors_data = user_neighbors.get_neighbors_data(user,itemID)
    #print(neighbors_data.shape)
    #print(neighbors_data)
    #similarity_of_neighbors = [similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]) for neighbor in neighbors_data]
    similarity_of_neighbors = []
    neighborhood_contribution_rating = []
    #print(neighbors_data.shape)
    for index, neighbor in neighbors_data.iterrows():
        similarity_of_neighbors.append(similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]))
        neighborhood_contribution_rating.append(user_neighborhood_contribution.neighbor_rating(neighbor,itemID,1,0.1) - user_neighborhood_contribution.neighbor_average(neighbor,1,0.1))
#     neighborhood_contribution_rating = [user_neighborhood_contribution.neighbor_rating(neighbor,item) - user_neighborhood_contribution.neighbor_average(neighbor)
#                                         for neighbor in neighbors_data]
    numerator_value = numerator(neighborhood_contribution_rating, similarity_of_neighbors)
    baseline_rating = user_neighborhood_contribution.baseline_rating(itemID)
    #print(numerator_value)
    #print(similarity_of_neighbors)
    #print(baseline_rating)
    final_rating = baseline_rating + numerator_value/ sum(similarity_of_neighbors)
    return final_rating

In [12]:
# if __name__ == "main":
#     userId = 20
#     iteId = 30
#     rating = predict_rating(20,30)
#     print(rating)

In [61]:
predict_rating(df.iloc[37], 47)
#df.iloc[5]"

C:\Users\Kunal\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


5.0479068684211841

In [14]:
#df[df.itemID == 47]

In [15]:
#df.iloc[37]

In [62]:
from sklearn.metrics import mean_squared_error

In [63]:
y_true = [df.iloc[i].rating for i in range(700,800)]

In [68]:
y_pred = [predict_rating(df.iloc[i], df.iloc[i].itemID) for i in range(700,800)]

In [19]:
#y_pred

In [65]:
mean_squared_error(y_true, y_pred)

0.80070837078709101

## Pyswarm Implementation

In [73]:
from pyswarm import pso

def banana(x):
    x1 = x[0]
    x2 = x[1]
    return x1**4 - 2*x2*x1**2 + x2**2 + x1**2 - 2*x1 + 5

def con(x):
    x1 = x[0]
    x2 = x[1]
    return [-(x1 + 0.25)**2 + 0.75*x2]

lb = [-3, -1]
ub = [2, 6]

xopt, fopt = pso(banana, lb, ub, f_ieqcons=con)

Stopping search: Swarm best position change less than 1e-08


In [74]:
df

,userID,itemID,age,sex,city,country,time,daytype,season,location,...,movieLanguage,movieYear,genre1,genre2,genre3,actor1,actor2,actor3,budget,rating
0,21,5,28,1,10,3,2,2,2,1,...,9,1998,7,6,10,1636,1539,1402,90000000,3
1,21,6,28,1,10,3,4,2,2,1,...,9,2008,7,10,18,1373,1510,1691,24000000,4
2,22,13,28,1,20,2,3,2,3,2,...,9,2010,1,14,19,1382,57,1109,60000000,4
3,21,7,28,1,10,3,4,2,2,1,...,9,2003,3,10,18,98,385,1827,35000000,3
4,21,15,28,1,10,3,3,1,2,1,...,9,1971,1,8,21,384,93,715,4000000,3
5,15,11,30,1,10,3,3,1,3,1,...,9,2010,7,18,18,862,61,1378,20000000,4
6,23,18,33,1,20,2,2,2,2,1,...,9,2010,1,3,21,1803,251,1376,82000000,4
7,27,19,25,1,10,3,2,1,2,1,...,14,2005,7,10,18,1635,869,1440,35000000,2
8,28,19,26,2,10,3,2,1,2,3,...,14,2005,7,10,18,1635,869,1440,35000000,2
9,29,20,30,1,20,2,4,1,3,1,...,9,2010,10,17,21,595,1542,1474,45000000,3
